In [4]:
!pip install -q kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

100% 2.29G/2.29G [00:30<00:00, 39.3MB/s]
100% 2.29G/2.29G [00:30<00:00, 80.2MB/s]


In [6]:
from zipfile import ZipFile
file_name='/content/chest-xray-pneumonia.zip'
with ZipFile(file_name,'r') as zip:
  zip.extractall()

In [7]:
import keras 
from keras.models import * 
from keras.layers import * 
from keras.preprocessing import image
import PIL
from keras.preprocessing.image import ImageDataGenerator

In [8]:
train_datagen = ImageDataGenerator( 
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)


test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        r'/content/chest_xray/train',
        color_mode = 'grayscale',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary', shuffle=True)

test_set = test_datagen.flow_from_directory(
        r'/content/chest_xray/test',
        color_mode = 'grayscale',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary', shuffle=True)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
# AlexNet
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), strides=(1,1), padding='same', activation='relu', input_shape=(224,224,1)))
model.add(Conv2D(64, kernel_size=(3, 3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), strides=(1,1), padding='same', activation='relu'))
model.add(Conv2D(128, kernel_size=(3, 3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))


model.add(Conv2D(256, kernel_size=(3, 3), strides=(1,1), padding='same', activation='relu'))
model.add(Conv2D(256, kernel_size=(3, 3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

model.add(Conv2D(512, kernel_size=(3, 3), strides=(1,1), padding='same', activation='relu'))
model.add(Conv2D(512, kernel_size=(3, 3), strides=(1,1), padding='same', activation='relu'))
model.add(Conv2D(512, kernel_size=(3, 3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

model.add(Conv2D(512, kernel_size=(3, 3), strides=(1,1), padding='same', activation='relu'))
model.add(Conv2D(512, kernel_size=(3, 3), strides=(1,1), padding='same', activation='relu'))
model.add(Conv2D(512, kernel_size=(3, 3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy, optimizer="adam", metrics=['accuracy'])


model.summary()

In [ ]:
#We'll try to train with 100 Epochs
results=model.fit( 
    training_set,
    batch_size=32,  
    epochs=30,
    validation_data=test_set,
    validation_steps=1
)

Epoch 1/30
163/163 [==============================] - 92s 369ms/step - loss: 0.6034 - accuracy: 0.7360 - val_loss: 0.8073 - val_accuracy: 0.5000
Epoch 2/30
163/163 [==============================] - 60s 369ms/step - loss: 0.5711 - accuracy: 0.7429 - val_loss: 0.6550 - val_accuracy: 0.6562
Epoch 3/30
163/163 [==============================] - 60s 367ms/step - loss: 0.5873 - accuracy: 0.7337 - val_loss: 0.6812 - val_accuracy: 0.3750
Epoch 4/30
163/163 [==============================] - 59s 364ms/step - loss: 0.5611 - accuracy: 0.7465 - val_loss: 0.7000 - val_accuracy: 0.4688
Epoch 5/30
163/163 [==============================] - 59s 364ms/step - loss: 0.4803 - accuracy: 0.7782 - val_loss: 0.6982 - val_accuracy: 0.6250
Epoch 6/30
163/163 [==============================] - 59s 364ms/step - loss: 0.4500 - accuracy: 0.8044 - val_loss: 0.6632 - val_accuracy: 0.6875
Epoch 7/30
163/163 [==============================] - 60s 364ms/step - loss: 0.3766 - accuracy: 0.8252 - val_loss: 0.5141 - val_ac

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.legend(['Training', 'Validation'])
plt.title('Training and Validation losses')
plt.xlabel('epoch')

In [ ]:
plt.plot(results.history['accuracy'])
plt.plot(results.history['val_accuracy'])
plt.legend(['Training', 'Validation'])
plt.title('Training and Validation accuracy')
plt.xlabel('epoch')
plt.ylim(0, 1);